In [ ]:
from numpy import array
from string import punctuation
from os import listdir
import nltk
nltk.download('stopwords')
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/TP1-NLP
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# update counts
	vocab.update(tokens)

# load all docs in a directory
def process_docs(directory, vocab):
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# add doc to vocab
		add_doc_to_vocab(path, vocab)

# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('txt_sentoken/pos', vocab)
process_docs('txt_sentoken/neg', vocab)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()

# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
# save tokens to a vocabulary file
save_list(tokens, 'vocab1.txt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive
/content/drive/My Drive/TP1-NLP


KeyboardInterrupt: ignored

In [ ]:
from numpy import array
from string import punctuation
from os import listdir
import nltk
nltk.download('stopwords')
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
    

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)

# load all docs in a directory
def process_docs(directory, vocab):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		'''if filename.startswith('cv9'):
			continue'''
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines

# load the vocabulary
vocab_filename = 'vocab1.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load all training reviews
def load_clean_dataset(vocab):
  pos_labels = list()
  neg_labels = list()
  # load all training reviews
  positive_lines = process_docs('txt_sentoken/pos', vocab)# print(type(positive_lines))
  for i in range(len(positive_lines)):pos_labels.append(1)
  negative_lines = process_docs('txt_sentoken/neg', vocab)
  for i in range(len(negative_lines)):neg_labels.append(0)
  docs = positive_lines + negative_lines
  labels = np.array(pos_labels + neg_labels)
  return docs, labels


In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
# integer encode and pad documents
def encode_docs(tokenizer, max_length, documents):
    # integer encode
    encoded = tokenizer.texts_to_sequences(documents)
    # pad sequence
    padded = pad_sequences(encoded, maxlen=max_length, 
             padding='post')
    return padded

# create the model
def create_model(vocabulary_size, max_length):
    # define network
    model = Sequential()
    model.add(Embedding(vocabulary_size, 100, 
               input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                  metrics=['accuracy'])
    # summarize defined model
    model.summary()
    return model


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from math import floor
from random import shuffle
import re
# load all reviews
docs, labels = load_clean_dataset(vocab)
# create the tokenizer
tokenizer = create_tokenizer(docs)
# split data for training and testing
split_limit = 1800
train_docs, ytrain = docs[:split_limit], labels[:split_limit]
test_docs, ytest = docs[split_limit:], labels[split_limit:]
# calculate maximum sequence length
max_length = max([len(s.split()) for s in docs])
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
Xtest = encode_docs(tokenizer, max_length, test_docs)
print(Xtrain.shape, Xtest.shape)
n_words = Xtest.shape[1]
# Vocabulary size
vocabulary_size = len(tokenizer.word_index)+1
# compile network

model = create_model(vocabulary_size, max_length)

# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
#ytest = np.array(ytest)
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

(1800, 1317) (200, 1317)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d (Conv1D)              (None, 1310, 32)          25632     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 655, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 20960)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                209610    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 2,812,053
Trainable params: 2,812,053
Non-trainable params: 0
_____________________

# Glove

In [33]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
 values = line.split()
 word = values[0]
 coefs = np.asarray(values[1:], dtype='float32')
 embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index)) 
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, i in tokenizer.word_index.items():
 embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None:
   embedding_matrix[i] = embedding_vector 

# create the model
def create_model(vocabulary_size, max_length, weights):
    # define network
    model = Sequential()
    model.add(Embedding(vocabulary_size, 100, weights = [weights],input_length=max_length, trainable=False))
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                  metrics=['accuracy'])
    # summarize defined model
    model.summary()
    return model

model = create_model(vocabulary_size, max_length, embedding_matrix)

# fit network
model.fit(Xtrain, ytrain, epochs=20, verbose=2)
# evaluate
ytest = np.array(ytest)
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Loaded 400000 word vectors.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1310, 32)          25632     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 655, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 20960)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                209610    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 2,812,053
Trainable params: 235,253
Non-trainable params: 2,576,800
__________